In [5]:
import tensorflow as tf
import tensorlayer as tl

tf.logging.set_verbosity(tf.logging.DEBUG)
tl.logging.set_verbosity(tl.logging.DEBUG)

In [14]:
tf.reset_default_graph()

In [15]:
sess = tf.InteractiveSession()

# prepare data
X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=(-1, 784))
# define placeholder
x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.int64, shape=[None], name='y_')

# define the network
network = tl.layers.InputLayer(x, name='input')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, 800, tf.nn.relu, name='relu1')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, 800, tf.nn.relu, name='relu2')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
# the softmax is implemented internally in tl.cost.cross_entropy(y, y_) to
# speed up computation, so we use identity here.
# see tf.nn.sparse_softmax_cross_entropy_with_logits()
network = tl.layers.DenseLayer(network, n_units=10, act=None, name='output')

# define cost function and metric.
y = network.outputs
cost = tl.cost.cross_entropy(y, y_, name='cost')
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

# define the optimizer
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost, var_list=train_params)

# initialize all variables in the session
tl.layers.initialize_global_variables(sess)

# print network information
network.print_params()
network.print_layers()

# train the network
tl.utils.fit(sess, network, train_op, cost, X_train, y_train, x, y_, acc=acc, batch_size=500, \
    n_epoch=50, print_freq=5, X_val=X_val, y_val=y_val, eval_train=False)

# evaluation
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

# save the network to .npz file
# tl.files.save_npz(network.all_params, name='model.npz')

[TL] Load or Download MNIST > data/mnist
[TL] data/mnist/train-images-idx3-ubyte.gz
[TL] data/mnist/t10k-images-idx3-ubyte.gz
[TL] InputLayer  input: (?, 784)
[TL] DropoutLayer drop1: keep: 0.800000 is_fix: False
[TL] DenseLayer  relu1: 800 relu
[TL] DropoutLayer drop2: keep: 0.500000 is_fix: False
[TL] DenseLayer  relu2: 800 relu
[TL] DropoutLayer drop3: keep: 0.500000 is_fix: False
[TL] DenseLayer  output: 10 No Activation
[TL]   param   0: relu1/W:0            (784, 800)         float32_ref (mean: -1.6607247744104825e-05, median: -0.0001256358518730849, std: 0.0880342349410057)   
[TL]   param   1: relu1/b:0            (800,)             float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   2: relu2/W:0            (800, 800)         float32_ref (mean: 8.036911458475515e-05, median: 0.0002151105145458132, std: 0.08790813386440277)   
[TL]   param   3: relu2/b:0            (800,)             float32_ref (mean: 0.0               ,

In [ ]:
tl.utils.fit()

In [16]:
network.print_params()

[TL]   param   0: relu1/W:0            (784, 800)         float32_ref (mean: -0.004262140486389399, median: -0.004270689561963081, std: 0.08819250762462616)   
[TL]   param   1: relu1/b:0            (800,)             float32_ref (mean: -0.010709851048886776, median: -0.010877817869186401, std: 0.01581108197569847)   
[TL]   param   2: relu2/W:0            (800, 800)         float32_ref (mean: -0.005490759387612343, median: -0.005541500169783831, std: 0.08763084560632706)   
[TL]   param   3: relu2/b:0            (800,)             float32_ref (mean: -0.01612836867570877, median: -0.016801461577415466, std: 0.019056975841522217)   
[TL]   param   4: output/W:0           (800, 10)          float32_ref (mean: -0.0011973264627158642, median: 0.0010191171895712614, std: 0.0809592604637146)   
[TL]   param   5: output/b:0           (10,)              float32_ref (mean: -0.0021968409419059753, median: -0.01207673642784357, std: 0.029742857441306114)   
[TL]   num of params: 1276810


In [10]:
sess.close()